In [13]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
import datetime
import pytz
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
from sqlalchemy.sql import func

class Base(DeclarativeBase):
    pass

class Word(Base):
    __tablename__ = "word_dict"
    id: Mapped[int] = mapped_column(primary_key=True)
    word: Mapped[str] 
    tonic_accent: Mapped[str]
    comment: Mapped[Optional[str]]
    index_ref_entries: Mapped[List["IndexEntry"]] = relationship(
        back_populates="word", cascade="all, delete-orphan"
    )
    def __repr__(self) -> str:
        return f"Word(id={self.id!r}, word={self.word!r}, comment={self.comment!r})"

class IndexEntry(Base):
    __tablename__ = "word_index"
    id : Mapped[int] = mapped_column(primary_key=True)
    lesson : Mapped[int]
    line : Mapped[int]
    position : Mapped[Optional[int]]
    word_id : Mapped[int] = mapped_column(ForeignKey("word_dict.id"))
    word : Mapped["Word"] = relationship(back_populates="index_ref_entries")
    def __repr__(self) -> str:
        return f"IndexEntry(id={self.id!r}, word={self.word!r}, word_id={self.word_id}, lesson={self.lesson!r}, line={self.line!r}), position={self.position!r}"
        
class Sentence(Base):
    __tablename__ = "sentences"
    id: Mapped[int] = mapped_column(primary_key=True)
    sentence: Mapped[str] 
    comment: Mapped[Optional[str]]
    lesson : Mapped[int]
    line : Mapped[int]
    def __repr__(self) -> str:
        return f"Sentence(id={self.id!r}, lesson={lesson!r}, numero={numero!r}, sentence ={self.sentence!r}, comment={self.comment!r})"

class MarkedSentence(Base):
    __tablename__ = "marked_sentences"
    id: Mapped[int] = mapped_column(primary_key=True)
    date_time: Mapped[datetime.datetime]
    sentence: Mapped[str] 
    comment: Mapped[Optional[str]]
    lesson : Mapped[int]
    line : Mapped[int]
    def __repr__(self) -> str:
        return f"Sentence(id={self.id!r}, date_time = {self.date_time!r}, lesson={self.lesson!r}, line={self.line!r}, sentence ={self.sentence!r}, comment={self.comment!r})"


In [14]:
from pathlib import Path

db_directory = "./db"
word_dict_filename = "test_tonic_accent_word_dict.db"

db_name = Path(db_directory, word_dict_filename)
db_url  = f"sqlite:///{db_name}"
db_url

'sqlite:///db/test_tonic_accent_word_dict.db'

In [15]:
from sqlalchemy import create_engine
engine = create_engine(db_url, echo=True)

In [16]:
Base.metadata.create_all(engine)

2024-09-02 12:24:11,304 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-02 12:24:11,305 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_dict")
2024-09-02 12:24:11,306 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-02 12:24:11,309 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_index")
2024-09-02 12:24:11,310 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-02 12:24:11,311 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sentences")
2024-09-02 12:24:11,312 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-02 12:24:11,313 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("marked_sentences")
2024-09-02 12:24:11,314 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-02 12:24:11,315 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
from time import time
ti = time()
from tonic_accent import global_word_dict as word_dict
tf = time()
print(f"time to fetch word dictionary from html files : {tf-ti}")


2024-08-31 10:59:07,907 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-31 10:59:07,907 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_dict")
2024-08-31 10:59:07,908 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-31 10:59:07,908 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("word_index")
2024-08-31 10:59:07,908 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-31 10:59:07,909 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sentences")
2024-08-31 10:59:07,909 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-31 10:59:07,910 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("marked_sentences")
2024-08-31 10:59:07,910 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-31 10:59:07,910 INFO sqlalchemy.engine.Engine COMMIT
time to fetch word dictionary from html files : 0.22655296325683594


In [6]:
keys_number = 0
for key in word_dict:
    keys_number += 1
print(f"Number of keys : {keys_number}")

Number of keys : 3555


In [7]:
word_dict["casa"]

[('ca', True), ('sa', False)]

In [8]:
word_dict["saldríamos"]

[('sal', False), ('drí', True), ('amos', False)]

In [9]:
word_dict["niños"]

[('ni', True), ('ños', False)]

In [7]:
from tonic_accent import get_word_index_dict

In [8]:
word_index_dict = get_word_index_dict()

In [17]:
from sqlalchemy.orm import Session

In [12]:
word_index_dict["ocurría".lower()]

[(48, 8)]

In [ ]:
list(word_index_dict.keys()

In [13]:
engine.echo=True

In [14]:
test_word_add_all_with_3_words = False
engine.echo=False
from sqlalchemy.orm import Session
import pickle
with Session(engine) as session:
    casa = Word(
        word="casa", 
        tonic_accent=pickle.dumps(word_index_dict["casa"]),
        index_ref_entries=[
            IndexEntry(lesson=9, line=5),
            IndexEntry(lesson=10, line=2),
            IndexEntry(lesson=10, line=3),
            IndexEntry(lesson=13, line=6),
            IndexEntry(lesson=13, line=14),
        ],
    )
    saldriamos = Word(
        word="saldríamos", 
        tonic_accent=pickle.dumps(word_index_dict["saldríamos"]),
        index_ref_entries=[
            IndexEntry(lesson=80, line=3),
        ]
    )
    ninos = Word(
        word="niños", 
        tonic_accent=pickle.dumps(word_index_dict["niños"]),
        index_ref_entries=[
            IndexEntry(lesson=24, line=10),
            IndexEntry(lesson=30, line=6),
            IndexEntry(lesson=30, line=15),
            IndexEntry(lesson=62, line=21),
            IndexEntry(lesson=79, line=12),
        ],
    )
    if test_word_add_all_with_3_words:
        session.add_all([casa, saldriamos, ninos])
        session.commit()
    
    


In [15]:
from sqlalchemy import select
stmt = select(Word).where(Word.word == "niños")

for word in session.scalars(stmt):
    print(word)

In [16]:
from sqlalchemy.orm import Session
import pickle
with Session(engine) as session:

    stmt = select(IndexEntry).where(IndexEntry.lesson == 62)

    for entry in session.scalars(stmt):
        print(entry)

In [17]:
stmt = select(IndexEntry).where(IndexEntry.word_id == 3)

for entry in session.scalars(stmt):
    print(entry)

In [18]:
from sqlalchemy.orm import Session
import pickle
engine.echo=False
ti = time()
with Session(engine) as session:
    for word in word_dict:
        entries = []
        for entry in word_index_dict[word]:
            lesson, line = entry
            entries.append(IndexEntry(lesson=lesson, line=line))
        word_obj = Word(
            word=word, 
            tonic_accent=pickle.dumps(word_dict[word]),
            index_ref_entries=entries
        )
        session.add_all([word_obj])
    session.commit()
tf = time()

print(f"time to store word_dict in db : {tf - ti}")


time to store word_dict in db : 1.1030960083007812


In [19]:
from sqlalchemy import select
stmt = select(Word).where(Word.word == "niños")

for word in session.scalars(stmt):
    print(word)
    for entry in word.index_ref_entries:
        print(f"lesson : {entry.lesson}, line : {entry.line}")

Word(id=639, word='niños', comment=None)
lesson : 24, line : 10
lesson : 30, line : 6
lesson : 30, line : 15
lesson : 62, line : 21
lesson : 79, line : 12
lesson : 80, line : 9
lesson : 80, line : 10
lesson : 80, line : 19


In [20]:
engine.echo=True
stmt = select(IndexEntry).where(IndexEntry.word_id == 639)

for entry in session.scalars(stmt):
    print(entry)

IndexEntry(id=13226, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=24, line=10), position=None
IndexEntry(id=13227, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=30, line=6), position=None
IndexEntry(id=13228, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=30, line=15), position=None
IndexEntry(id=13229, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=62, line=21), position=None
IndexEntry(id=13230, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=79, line=12), position=None
IndexEntry(id=13231, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=80, line=9), position=None
IndexEntry(id=13232, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=80, line=10), position=None
IndexEntry(id=13233, word=Word(id=639, word='niños', comment=None), word_id=639, lesson=80, line=19), position=None


In [21]:
word_index_dict["niños"]

[(24, 10), (30, 6), (30, 15), (62, 21), (79, 12), (80, 9), (80, 10), (80, 19)]

In [22]:
from sqlalchemy import select
stmt = select(Word).where(Word.word == "quedaran")

for word in session.scalars(stmt):
    print(word)

Word(id=2627, word='quedaran', comment=None)


In [23]:
stmt = select(IndexEntry).where(IndexEntry.word_id == 2627)

for entry in session.scalars(stmt):
    print(entry)

IndexEntry(id=18271, word=Word(id=2627, word='quedaran', comment=None), word_id=2627, lesson=80, line=9), position=None


In [ ]:
stmt = select(Word)
new_word_index_dict = {}
ti = time()
for word in session.scalars(stmt):
    index_ref = []
    for entry in word.index_ref_entries:
        index_ref.append((entry.lesson, entry.line))
    new_word_index_dict[word.word] = index_ref
tf = time()
print(f"time to fetch index word dictionary from db : {tf-ti}")

2024-09-02 12:30:58,286 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-02 12:30:58,290 INFO sqlalchemy.engine.Engine SELECT word_dict.id, word_dict.word, word_dict.tonic_accent, word_dict.comment 
FROM word_dict
2024-09-02 12:30:58,291 INFO sqlalchemy.engine.Engine [generated in 0.00101s] ()
2024-09-02 12:30:58,322 INFO sqlalchemy.engine.Engine SELECT word_index.id AS word_index_id, word_index.lesson AS word_index_lesson, word_index.line AS word_index_line, word_index.position AS word_index_position, word_index.word_id AS word_index_word_id 
FROM word_index 
WHERE ? = word_index.word_id
2024-09-02 12:30:58,322 INFO sqlalchemy.engine.Engine [generated in 0.00050s] (1,)
2024-09-02 12:30:58,329 INFO sqlalchemy.engine.Engine SELECT word_index.id AS word_index_id, word_index.lesson AS word_index_lesson, word_index.line AS word_index_line, word_index.position AS word_index_position, word_index.word_id AS word_index_word_id 
FROM word_index 
WHERE ? = word_index.word_id
2024-09-02 12:

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2024-09-02 12:31:01,977 INFO sqlalchemy.engine.Engine [cached since 3.656s ago] (2631,)
2024-09-02 12:31:01,978 INFO sqlalchemy.engine.Engine SELECT word_index.id AS word_index_id, word_index.lesson AS word_index_lesson, word_index.line AS word_index_line, word_index.position AS word_index_position, word_index.word_id AS word_index_word_id 
FROM word_index 
WHERE ? = word_index.word_id
2024-09-02 12:31:01,978 INFO sqlalchemy.engine.Engine [cached since 3.657s ago] (2632,)
2024-09-02 12:31:01,979 INFO sqlalchemy.engine.Engine SELECT word_index.id AS word_index_id, word_index.lesson AS word_index_lesson, word_index.line AS word_index_line, word_index.position AS word_index_position, word_index.word_id AS word_index_word_id 
FROM word_index 
WHERE ? = word_index.word_id
2024-09-02 12:31:01,979 INFO sqlalchemy.engine.Engine [cached since 3.658s ago] (2633,)
2024-09-02 12:31:01,980 INFO sqlalchemy.engine.Engine SELECT word_index.id AS word_index_id, word_index.lesson AS word_index_lesson, w

In [ ]:
new_word_index_dict["leer"]

In [22]:
new_word_index_dict = {}
stmt = select(IndexEntry)
ti = time()
with Session(engine) as session:
    for entry in session.scalars(stmt):
        if not entry.word.word in new_word_index_dict:
            new_word_index_dict[entry.word.word] = [(entry.lesson, entry.line)]
        else:
            new_word_index_dict[entry.word.word].append((entry.lesson, entry.line))
tf = time()
print(f"time to fetch index word dictionary from db directly: {tf-ti}")    

2024-09-02 12:27:40,402 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-02 12:27:40,404 INFO sqlalchemy.engine.Engine SELECT word_index.id, word_index.lesson, word_index.line, word_index.position, word_index.word_id 
FROM word_index
2024-09-02 12:27:40,405 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ()
2024-09-02 12:27:40,588 INFO sqlalchemy.engine.Engine SELECT word_dict.id AS word_dict_id, word_dict.word AS word_dict_word, word_dict.tonic_accent AS word_dict_tonic_accent, word_dict.comment AS word_dict_comment 
FROM word_dict 
WHERE word_dict.id = ?
2024-09-02 12:27:40,589 INFO sqlalchemy.engine.Engine [generated in 0.00053s] (1,)
2024-09-02 12:27:40,589 INFO sqlalchemy.engine.Engine SELECT word_dict.id AS word_dict_id, word_dict.word AS word_dict_word, word_dict.tonic_accent AS word_dict_tonic_accent, word_dict.comment AS word_dict_comment 
FROM word_dict 
WHERE word_dict.id = ?
2024-09-02 12:27:40,590 INFO sqlalchemy.engine.Engine [cached since 0.001409s ago] (2,)


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2024-09-02 12:27:43,959 INFO sqlalchemy.engine.Engine [cached since 3.371s ago] (3394,)
2024-09-02 12:27:43,960 INFO sqlalchemy.engine.Engine SELECT word_dict.id AS word_dict_id, word_dict.word AS word_dict_word, word_dict.tonic_accent AS word_dict_tonic_accent, word_dict.comment AS word_dict_comment 
FROM word_dict 
WHERE word_dict.id = ?
2024-09-02 12:27:43,960 INFO sqlalchemy.engine.Engine [cached since 3.372s ago] (3395,)
2024-09-02 12:27:43,960 INFO sqlalchemy.engine.Engine SELECT word_dict.id AS word_dict_id, word_dict.word AS word_dict_word, word_dict.tonic_accent AS word_dict_tonic_accent, word_dict.comment AS word_dict_comment 
FROM word_dict 
WHERE word_dict.id = ?
2024-09-02 12:27:43,960 INFO sqlalchemy.engine.Engine [cached since 3.372s ago] (3396,)
2024-09-02 12:27:43,961 INFO sqlalchemy.engine.Engine SELECT word_dict.id AS word_dict_id, word_dict.word AS word_dict_word, word_dict.tonic_accent AS word_dict_tonic_accent, word_dict.comment AS word_dict_comment 
FROM word_dic

In [27]:
new_word_index_dict["casa"]

[(9, 5),
 (10, 2),
 (10, 3),
 (13, 6),
 (13, 14),
 (14, 5),
 (26, 3),
 (26, 15),
 (28, 7),
 (28, 11),
 (35, 9),
 (78, 2),
 (78, 20),
 (80, 12),
 (84, 8),
 (88, 4),
 (88, 7),
 (90, 1)]

In [28]:
stmt = select(Word)
new_dict = {}
ti = time()
for word in session.scalars(stmt):
    new_dict[word.word] = pickle.loads(word.tonic_accent)
tf = time()
print(f"time to fetch word dictionary with only tonic_accent from db : {tf-ti}")

time to fetch word dictionary with only tonic_accent from db : 0.031355857849121094


In [29]:
new_dict["casa"]

[('ca', True), ('sa', False)]

In [30]:
f = open('testdummy.dat', 'wb')
ti = time()
pickle.dump(word_dict, f)
tf = time()
print(f"time to write word dictionnary to pickle file {tf-ti}s")

time to write word dictionnary to pickle file 0.005590200424194336s


In [31]:
f = open('testdummy.dat', 'rb')
ti = time()
pickle.load(f)
tf = time()
print(f"time to read word dictionnary from pickle file {tf-ti}s")

time to read word dictionnary from pickle file 0.007337093353271484s


In [32]:
from lesson_parser import get_words
from tonic_accent import get_sentences

ti = time()
from sqlalchemy.orm import Session
engine.echo=False
with Session(engine) as session:
    for lesson in range(1, 101):
        sentences = get_sentences(lesson)
        for line, sentence in enumerate(sentences):
            s_obj = Sentence(
                lesson = lesson,
                line = line,
                sentence = sentence
            )
            session.add(s_obj)
    session.commit()

tf = time()
print(f"time to write all sentences to db : {tf-ti}")

time to write all sentences to db : 0.15726518630981445


In [21]:
from sqlalchemy import select, and_
stmt = select(Sentence).where(and_(Sentence.line == 11, Sentence.lesson == 50))
ti = time()
with Session(engine) as session:
    for s_obj in session.scalars(stmt):
        print(s_obj.lesson, s_obj.line, s_obj.sentence)
tf = time()
print(f"time to fetch word dictionary from db : {tf-ti}")

2024-09-02 12:27:12,468 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-02 12:27:12,485 INFO sqlalchemy.engine.Engine SELECT sentences.id, sentences.sentence, sentences.comment, sentences.lesson, sentences.line 
FROM sentences 
WHERE sentences.line = ? AND sentences.lesson = ?
2024-09-02 12:27:12,486 INFO sqlalchemy.engine.Engine [generated in 0.00099s] (11, 50)
50 11 S10-¡Ay, cuánto se lo agradezco! Es usted muy amable. 
2024-09-02 12:27:12,490 INFO sqlalchemy.engine.Engine ROLLBACK
time to fetch word dictionary from db : 0.026129961013793945


In [34]:
word = "ganas"
stmt = select(Word).where(Word.word == word)
word_obj = session.scalars(stmt).one()
print(word_obj)
for entry in word_obj.index_ref_entries:
    stmt = select(Sentence).where(and_(Sentence.lesson == entry.lesson, Sentence.line == entry.line))
    for s_obj in session.scalars(stmt):
        print(f"{s_obj.lesson}, {s_obj.line} : {s_obj.sentence}")
    

Word(id=326, word='ganas', comment=None)
13, 5 : S04-Lo siento, yo tampoco tengo ganas de ir de copas. 
13, 13 : T04-Mi mujer tampoco tiene ganas de ir de copas. 
14, 3 : S02-No, no tengo ganas de hacer nada. 
25, 8 : S07-Y si tengo ganas de mejor tiempo, 
30, 2 : S01-Nochebuena, Nochevieja, Reyes ¡Tengo ganas de que acaben estas fiestas! 
30, 12 : T01-Tengo ganas de que los Reyes Magos me traigan regalos el seis de enero. 
35, 9 : S08-No tengo mucha hambre ni muchas ganas de beber Creo que me voy a quedar en casa. 
35, 11 : S10-En absoluto, lo siento. Solo tengo ganas de algo ligero un puré de verduras, agua con gas y dormir. 
49, 4 : S03-La gente se vuelve vaga nadie tiene ganas de ir por un sello al estanco o por un sobre a la papelería. 
59, 15 : S14-Venga cara ganas tú, cruz gano yo. 
89, 18 : S17-¿A que te mueres de ganas de probarla? 


In [35]:
def store_lesson_error(lesson_number, sentences, marked_lines_numbers):
    with Session(engine) as session:
        for line, s in enumerate(sentences):
            if line in marked_lines_numbers:
                s_obj = MarkedSentence(
                    lesson = lesson_number,
                    line = line,
                    date_time = datetime.datetime.now(),
                    sentence = sentences[line]
                )
                session.add(s_obj)
        session.commit()

                    
                
    

In [36]:
sentences = ['N4-Lec<b>ción</b> <b>cua</b>rta',
 'S00-TITLE-Prese<mark>nta<b>cio</b>n</mark>es',
 'S01-Él es <b>Ra</b>fa, <mark>mi her<b>ma</b>no, y <b>e</b>lla es Luz, mi her<b>ma</b>na.  </mark>',
 '<mark> S02-Y él es mi <b>pa</b>dre, </mark>Mi<b>guel</b>.',
 'S03-<b>Mu</b>cho <b>gus</b>to. ¿<b>Có</b>mo es<b>tá</b> us<b>ted</b>?',
 'S04-Muy bien, <b>pe</b>ro ¡<b>h</b><mark><b>á</b>blame d</mark>e tú, por fa<b>vor</b>!',
 'S05-¡<b>Va</b>le!',
 'T00-TRANSLA<mark>TE-Ejer<b>ci</b>cio </mark>1 – Tra<b>duz</b>ca',
 'T01-Él es mi <b>pa</b>dre.',
 'T02-Y <b>e</b>lla es mi her<b>ma</b>na.',
 'T03-<b>Mu</b>cho <b></b><mark><b>gus</b>to.  </mark>',
 '<mark> T04-<b>Há</b>blame de </mark>tú, por fa<b>vor</b>.',
 'T05-¿<b>Có</b>mo es<b>tá</b> us<b>ted</b>?']

sentences

['N4-Lec<b>ción</b> <b>cua</b>rta',
 'S00-TITLE-Prese<mark>nta<b>cio</b>n</mark>es',
 'S01-Él es <b>Ra</b>fa, <mark>mi her<b>ma</b>no, y <b>e</b>lla es Luz, mi her<b>ma</b>na.  </mark>',
 '<mark> S02-Y él es mi <b>pa</b>dre, </mark>Mi<b>guel</b>.',
 'S03-<b>Mu</b>cho <b>gus</b>to. ¿<b>Có</b>mo es<b>tá</b> us<b>ted</b>?',
 'S04-Muy bien, <b>pe</b>ro ¡<b>h</b><mark><b>á</b>blame d</mark>e tú, por fa<b>vor</b>!',
 'S05-¡<b>Va</b>le!',
 'T00-TRANSLA<mark>TE-Ejer<b>ci</b>cio </mark>1 – Tra<b>duz</b>ca',
 'T01-Él es mi <b>pa</b>dre.',
 'T02-Y <b>e</b>lla es mi her<b>ma</b>na.',
 'T03-<b>Mu</b>cho <b></b><mark><b>gus</b>to.  </mark>',
 '<mark> T04-<b>Há</b>blame de </mark>tú, por fa<b>vor</b>.',
 'T05-¿<b>Có</b>mo es<b>tá</b> us<b>ted</b>?']

In [37]:
marked_lines_numbers = {1: True, 2: True, 3: True, 5: True, 7: True, 10: True, 11: True}

In [38]:
lesson_number = 4
lesson_number

4

In [39]:
store_lesson_error(lesson_number, sentences, marked_lines_numbers)

In [40]:
from sqlalchemy import select
stmt = select(MarkedSentence)

for marked_line in session.scalars(stmt):
    print(marked_line)

Sentence(id=1, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 279989), lesson=4, line=1, sentence ='S00-TITLE-Prese<mark>nta<b>cio</b>n</mark>es', comment=None)
Sentence(id=2, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 280221), lesson=4, line=2, sentence ='S01-Él es <b>Ra</b>fa, <mark>mi her<b>ma</b>no, y <b>e</b>lla es Luz, mi her<b>ma</b>na.  </mark>', comment=None)
Sentence(id=3, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 280261), lesson=4, line=3, sentence ='<mark> S02-Y él es mi <b>pa</b>dre, </mark>Mi<b>guel</b>.', comment=None)
Sentence(id=4, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 280295), lesson=4, line=5, sentence ='S04-Muy bien, <b>pe</b>ro ¡<b>h</b><mark><b>á</b>blame d</mark>e tú, por fa<b>vor</b>!', comment=None)
Sentence(id=5, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 280331), lesson=4, line=7, sentence ='T00-TRANSLA<mark>TE-Ejer<b>ci</b>cio </mark>1 – Tra<b>duz</b>ca', comment=None)
Sentence(id=6, date_time = dateti

In [41]:
sentences = ['N5-Lec<b>ción</b> <b>quin</b>ta',
 'S00-TITLE-Entre<b>vis</b>ta de tra<b>ba</b>jo',
 'S01-¿<b>Cu</b><mark><b>á</b>ntos i<b>dio</b>mas <b>ha</b>bla us<b>ted</b>?  </mark>',
 '<mark> S02-<b>Ha</b>blo in<b>glés</b> y fra</mark>n<b>cés</b>.',
 'S03-¡<mark>Enhora<b>bue</b>na! ¿Ha<b>bla</b>mos in<b>glés</b> en<b>ton</b>ces?  </mark>',
 '<mark> S04-E<b>jem</b>, el in<b>glés</b> lo <b>le</b>o, <b>pe</b>ro no lo <b>ha</b>blo muy bien.  </mark>',
 '<mark> S05-Sí, <b>cla</b>ro</mark> Y el fran<b>cés</b>, ¿lo <b>ha</b>bla o <b>so</b>lo lo es<b>cri</b>be?',
 'T00-TRANSLATE-Ejer<b>ci</b>cio 1 – Tra<b>duz</b>ca',
 'T01-¿<b>H</b><mark><b>a</b>bla us<b>ted</b> in<b>glés</b>?  </mark>',
 '<mark> T02-¿<b>Cuá</b>ntos i<b>dio</b>mas es<b>cri</b>be us<b>te</b></mark><b>d</b>?',
 'T03-No <b>ha</b>blo fran<b>cés</b>.',
 'T04-¡Enhora<b>bue</b>na!',
 'T05-Lo <b>le</b>o, <b>pe</b>ro no lo es<b>cri</b>bo.']

sentences

['N5-Lec<b>ción</b> <b>quin</b>ta',
 'S00-TITLE-Entre<b>vis</b>ta de tra<b>ba</b>jo',
 'S01-¿<b>Cu</b><mark><b>á</b>ntos i<b>dio</b>mas <b>ha</b>bla us<b>ted</b>?  </mark>',
 '<mark> S02-<b>Ha</b>blo in<b>glés</b> y fra</mark>n<b>cés</b>.',
 'S03-¡<mark>Enhora<b>bue</b>na! ¿Ha<b>bla</b>mos in<b>glés</b> en<b>ton</b>ces?  </mark>',
 '<mark> S04-E<b>jem</b>, el in<b>glés</b> lo <b>le</b>o, <b>pe</b>ro no lo <b>ha</b>blo muy bien.  </mark>',
 '<mark> S05-Sí, <b>cla</b>ro</mark> Y el fran<b>cés</b>, ¿lo <b>ha</b>bla o <b>so</b>lo lo es<b>cri</b>be?',
 'T00-TRANSLATE-Ejer<b>ci</b>cio 1 – Tra<b>duz</b>ca',
 'T01-¿<b>H</b><mark><b>a</b>bla us<b>ted</b> in<b>glés</b>?  </mark>',
 '<mark> T02-¿<b>Cuá</b>ntos i<b>dio</b>mas es<b>cri</b>be us<b>te</b></mark><b>d</b>?',
 'T03-No <b>ha</b>blo fran<b>cés</b>.',
 'T04-¡Enhora<b>bue</b>na!',
 'T05-Lo <b>le</b>o, <b>pe</b>ro no lo es<b>cri</b>bo.']

In [42]:
marked_lines_numbers = {2: True, 3: True, 4: True, 5: True, 6: True, 8: True, 9: True}

marked_lines_numbers

{2: True, 3: True, 4: True, 5: True, 6: True, 8: True, 9: True}

In [43]:
lesson_number = 5
lesson_number

5

In [44]:
store_lesson_error(lesson_number, sentences, marked_lines_numbers)

In [45]:
lesson_number = 6
lesson_number

6

In [46]:
marked_lines_numbers = {3: True, 8: True, 9: True, 10: True, 11: True}
marked_lines_numbers

{3: True, 8: True, 9: True, 10: True, 11: True}

In [47]:
sentences = ['N6-Lec<b>ción</b> <b>sex</b>ta',
 'S00-TITLE-A las <b>sie</b>te ¡de la ma<b>ña</b>na!',
 'S01-¿En qué tra<b>ba</b>jas?',
 'S02-Soy enfer<b>m</b><mark><b>e</b>ra, tra<b>ba</b>jo en el hospi<b>ta</b></mark><b>l</b>.',
 'S03-¿A qué <b>ho</b>ra ter<b>mi</b>nas?',
 'S04-Ter<b>mi</b>no a las <b>sie</b>te.',
 'S05-Pues te in<b>vi</b>to al <b>ci</b>ne a las <b>o</b>cho, ¿<b>va</b>le?',
 'S06-<b>Gra</b>cias, <b>pe</b>ro ter<b>mi</b>no a las <b>sie</b>te ¡de la ma<b>ña</b>na!',
 'ST00-TRANSLATE-Ejer<b>ci</b>cio 1 – Tra<b>d<mark>u</mark>z</b>ca',
 'ST01-¿<b>E</b>res <mark>enfer<b>me</b>ra?  </mark>',
 '<mark> ST02-¿Tra<b>ba</b>jas en el hospi<b>tal</b>?  </mark>',
 '<mark> ST03-No, tra<b>ba</b>jo en e</mark>l <b>ci</b>ne.',
 'ST04-Te in<b>vi</b>to a las <b>o</b>cho, ¿<b>va</b>le?',
 'ST05-¿Ter<b>mi</b>nas a las <b>sie</b>te?']

sentences

['N6-Lec<b>ción</b> <b>sex</b>ta',
 'S00-TITLE-A las <b>sie</b>te ¡de la ma<b>ña</b>na!',
 'S01-¿En qué tra<b>ba</b>jas?',
 'S02-Soy enfer<b>m</b><mark><b>e</b>ra, tra<b>ba</b>jo en el hospi<b>ta</b></mark><b>l</b>.',
 'S03-¿A qué <b>ho</b>ra ter<b>mi</b>nas?',
 'S04-Ter<b>mi</b>no a las <b>sie</b>te.',
 'S05-Pues te in<b>vi</b>to al <b>ci</b>ne a las <b>o</b>cho, ¿<b>va</b>le?',
 'S06-<b>Gra</b>cias, <b>pe</b>ro ter<b>mi</b>no a las <b>sie</b>te ¡de la ma<b>ña</b>na!',
 'ST00-TRANSLATE-Ejer<b>ci</b>cio 1 – Tra<b>d<mark>u</mark>z</b>ca',
 'ST01-¿<b>E</b>res <mark>enfer<b>me</b>ra?  </mark>',
 '<mark> ST02-¿Tra<b>ba</b>jas en el hospi<b>tal</b>?  </mark>',
 '<mark> ST03-No, tra<b>ba</b>jo en e</mark>l <b>ci</b>ne.',
 'ST04-Te in<b>vi</b>to a las <b>o</b>cho, ¿<b>va</b>le?',
 'ST05-¿Ter<b>mi</b>nas a las <b>sie</b>te?']

In [48]:
store_lesson_error(lesson_number, sentences, marked_lines_numbers)

In [49]:
stmt = select(MarkedSentence)

for marked_line in session.scalars(stmt):
    print(marked_line)

Sentence(id=1, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 279989), lesson=4, line=1, sentence ='S00-TITLE-Prese<mark>nta<b>cio</b>n</mark>es', comment=None)
Sentence(id=2, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 280221), lesson=4, line=2, sentence ='S01-Él es <b>Ra</b>fa, <mark>mi her<b>ma</b>no, y <b>e</b>lla es Luz, mi her<b>ma</b>na.  </mark>', comment=None)
Sentence(id=3, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 280261), lesson=4, line=3, sentence ='<mark> S02-Y él es mi <b>pa</b>dre, </mark>Mi<b>guel</b>.', comment=None)
Sentence(id=4, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 280295), lesson=4, line=5, sentence ='S04-Muy bien, <b>pe</b>ro ¡<b>h</b><mark><b>á</b>blame d</mark>e tú, por fa<b>vor</b>!', comment=None)
Sentence(id=5, date_time = datetime.datetime(2024, 8, 26, 14, 51, 0, 280331), lesson=4, line=7, sentence ='T00-TRANSLA<mark>TE-Ejer<b>ci</b>cio </mark>1 – Tra<b>duz</b>ca', comment=None)
Sentence(id=6, date_time = dateti

In [50]:
from datetime import timedelta
stmt = select(MarkedSentence)

history = []
for marked_line_entry in session.scalars(stmt):
    history.append(marked_line_entry)

tb = history[0].date_time + timedelta(0, 1)
te = history[-1].date_time - timedelta(0, 1)
te

datetime.datetime(2024, 8, 26, 14, 51, 16, 32192)

In [51]:
stmt = select(MarkedSentence).where(and_(MarkedSentence.date_time > tb, MarkedSentence.date_time < te))

for marked_line in session.scalars(stmt):
    print(marked_line)

Sentence(id=8, date_time = datetime.datetime(2024, 8, 26, 14, 51, 10, 833773), lesson=5, line=2, sentence ='S01-¿<b>Cu</b><mark><b>á</b>ntos i<b>dio</b>mas <b>ha</b>bla us<b>ted</b>?  </mark>', comment=None)
Sentence(id=9, date_time = datetime.datetime(2024, 8, 26, 14, 51, 10, 833949), lesson=5, line=3, sentence ='<mark> S02-<b>Ha</b>blo in<b>glés</b> y fra</mark>n<b>cés</b>.', comment=None)
Sentence(id=10, date_time = datetime.datetime(2024, 8, 26, 14, 51, 10, 833986), lesson=5, line=4, sentence ='S03-¡<mark>Enhora<b>bue</b>na! ¿Ha<b>bla</b>mos in<b>glés</b> en<b>ton</b>ces?  </mark>', comment=None)
Sentence(id=11, date_time = datetime.datetime(2024, 8, 26, 14, 51, 10, 834018), lesson=5, line=5, sentence ='<mark> S04-E<b>jem</b>, el in<b>glés</b> lo <b>le</b>o, <b>pe</b>ro no lo <b>ha</b>blo muy bien.  </mark>', comment=None)
Sentence(id=12, date_time = datetime.datetime(2024, 8, 26, 14, 51, 10, 834050), lesson=5, line=6, sentence ='<mark> S05-Sí, <b>cla</b>ro</mark> Y el fran<b>cés</

In [40]:
import datetime
import pytz
utc_dt = datetime.datetime.now(tz=pytz.utc)
amsterdam_tz = pytz.timezone("Europe/Paris")
local_amsterdam_time = amsterdam_tz.normalize(utc_dt)
print(local_amsterdam_time.hour)
print (local_amsterdam_time.minute)
print (local_amsterdam_time.second)
local_amsterdam_time.now()

7
33
45


datetime.datetime(2024, 8, 26, 7, 33, 45, 965149)